# Plotting models fitted with fastNeutrino

In [ ]:
def import_fastNeutrino_fit(fn):
    with open(fn) as f:
        n_anc = int(f.readline())
        params = []
        for line in f:
            if line.startswith('c'):
                pass
            elif line.startswith('e'):
                pass
            else:
                continue
        return n_anc, params

In [1]:
chroms = ['Chr' + x for x in ['2L,','2R','3L','3R']]

In [2]:
models = ['2EpochC', '2EpochE']

In [ ]:
for c in chroms:
    